In [1]:
!pip install --quiet -U langchain langchain-openai
%pip install --quiet --upgrade langchain langchain-community langchain-chroma


Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install --quiet supabase

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 13.1
    Uninstalling websockets-13.1:
      Successfully uninstalled websockets-13.1
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0


In [4]:
import os
from supabase import create_client, Client

openai_api_key = os.getenv("OPENAI_API_KEY_1010")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

print(f"OpenAI API: {openai_api_key[:4]}********")
print(f"LangChain API Key: {langchain_api_key[:4]}********")

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["LANGCHAIN_PROJECT"] = "tech4impact_1010"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"

SUPABASE_URL = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
SUPABASE_KEY = os.getenv("NEXT_PUBLIC_SUPABASE_SERVICE_KEY")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)



OpenAI API: sk-p********
LangChain API Key: lsv2********


LangSmith Setup (LLM Logging)


In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5618e33f-9827-463d-b0f0-7fad4af6f599-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

Starts from now

In [6]:
import os
import json
import getpass
import logging 
# Set your OpenAI API key
#os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA



In [7]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [8]:
def ms_to_minutes_seconds_str(milliseconds):
    seconds = milliseconds // 1000  # 총 초
    minutes = seconds // 60  # 분 계산
    remaining_seconds = seconds % 60  # 남은 초 계산
    return f"{minutes}분 {remaining_seconds}초"  # 출력 형식

# 파일 
with open('transcription.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

utterances = data['utterances']

n = 3941239

# 발화를 Document 객체로 생성
filtered_documents = [
    Document(
        page_content=utterance['msg'],
        metadata={
            'start_at': utterance['start_at'],
            'duration': utterance['duration'],
            'speaker': utterance['spk'],
            'speaker_type': utterance['spk_type']
        }
    )
    for utterance in utterances if utterance['start_at'] > n
]
logging.info(f"총 {len(filtered_documents)}개의 문서 생성.")

# 시작 시간 기준으로 정렬
documents = sorted(filtered_documents, key=lambda doc: doc.metadata['start_at'])


2024-11-08 15:24:23,647 - INFO - 총 403개의 문서 생성.


In [ ]:
full_text = ''
for doc in documents:
    start_time_str = ms_to_minutes_seconds_str(doc.metadata['start_at'])
    full_text += f"[{start_time_str}] {doc.page_content}\n"


# 언어 모델 초기화 # mini 버전 써보기
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)



# 임베딩 및 벡터스토어 초기화
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings, persist_directory=None)

# 리트리버 
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


C:\Users\yooji\AppData\Local\Temp\ipykernel_21116\3297828087.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
C:\Users\yooji\AppData\Local\Temp\ipykernel_21116\3297828087.py:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings()
2024-11-08 15:24:32,733 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for

In [10]:
import re
# 역할 지정해서 돌려보기
#role description
#확정은 결과물 가지고 펠로우님이랑 이야기
#chunk 늘려보기..
prompt = """
        당신은 전문적인 상담 세션을 분석하고 요약하는 AI입니다. 아래 상담 세션의 전체 스크립트를 주제별로 시간 순서에 따라 요약해 주세요. 다음 지침을 반드시 따르세요:
        
        1. 각 시간 구간은 **하나의 주요 질문** 또는 **하나의 주요 조언**으로 구성합니다. 하나의 구간에 여러 주제가 섞이지 않도록 주의해 주세요.
        2. 각 주제에 대해 시작 시간과 종료 시간을 명시합니다. 시작 시간은 해당 주제가 스크립트에서 처음 등장한 시점, 종료 시간은 마지막으로 등장한 시점을 기준으로 합니다.
        3. 각 요약은 다음 형식 중 **하나**로 작성합니다:
           - 질문인 경우: **Q.**으로 시작하여 질문 내용을 요약합니다.
           - 행동 변경이 필요한 경우: **[바꾸세요]**로 시작하여 변경이 필요한 행동을 요약합니다.
           - 권장 사항인 경우: **[이렇게 하세요]**로 시작하여 권장 사항을 요약합니다.
        4. 스크립트에는 의약 관련, 생활 습관 등이 아닌 상담과 관련 없는 대화 내용이 일부 포함되어 있습니다. 그런 경우, 주제 산정에서 제외해주세요.


        **답변 예시:**
        **0분 42초 ~ 3분 15초**
        - Q. 오메가3 가 정말 유익한 거 맞나요?
        
        **5분 0초 ~ 7분 30초**
        - [바꾸세요] 당뇨 약은 중단하지 마세요.
        
        **10분 15초 ~ 12분 45초**
        - [이렇게 하세요] 약 복용을 지속하세요.
        
        **스크립트:**
        {text}
        
        **요약:**
"""

human_prompt = PromptTemplate(
    input_variables=[],
    template="요약:"
)


rag_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",  # 또는 "map_reduce" 등 필요에 따라 변경 가능
            retriever=retriever,
            return_source_documents=True
        )

# 전체 스크립트 요약 
response = rag_chain({"query": prompt.format(text=full_text)})
session_summary = response["result"]
source_documents = response["source_documents"]

print("=== Session Summary ===")
print(session_summary)
# print("\n=== Source Documents ===")
# for doc in source_documents:
#     print(f"Source: {doc.page_content}\n")


C:\Users\yooji\AppData\Local\Temp\ipykernel_21116\1108946316.py:48: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = rag_chain({"query": prompt.format(text=full_text)})
2024-11-08 15:26:40,892 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:26:51,960 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Session Summary ===
**66분 8초 ~ 66분 42초**
- Q. 고지혈증 약을 끊었는데, 영양제를 먹어야 할까요?

**67분 4초 ~ 67분 19초**
- [이렇게 하세요] 간 수치가 높지 않다면, 영양제를 드셔도 괜찮습니다.

**69분 10초 ~ 69분 15초**
- [이렇게 하세요] 피로감이 있다면, 영양제를 드시는 것이 좋습니다.

**69분 29초 ~ 70분 40초**
- Q. 병원에서 처방받은 약과 약국에서 구입한 약의 차이는 무엇인가요?

**71분 5초 ~ 71분 10초**
- [바꾸세요] 감기 초기에 비타민을 섭취하여 면역력을 높이세요.

**72분 1초 ~ 72분 17초**
- [이렇게 하세요] 감기약은 증상에 맞춰서 드세요.

**74분 2초 ~ 74분 22초**
- [이렇게 하세요] 열이 나면 타이레놀을 드세요.

**87분 48초 ~ 88분 26초**
- [이렇게 하세요] 고지혈증 약을 오래 복용했다면 코엔자임 큐텐을 보충하세요.

**89분 10초 ~ 89분 19초**
- [이렇게 하세요] 저녁 식사를 간단히 하고 일찍 주무세요.

**92분 8초 ~ 92분 50초**
- [이렇게 하세요] 저녁 식사는 탄수화물보다 단백질과 채소 위주로 드세요.

**93분 0초 ~ 93분 34초**
- [바꾸세요] 술로 인해 잠을 자는 것은 피하세요.

**94분 8초 ~ 94분 49초**
- [이렇게 하세요] 잠들기 전에는 소음 없이 주무세요.

**97분 0초 ~ 97분 11초**
- [이렇게 하세요] 주말에는 낮잠을 통해 피로를 푸세요.

**98분 21초 ~ 98분 34초**
- [이렇게 하세요] 필요에 따라 영양제를 추천받아 드세요.

**99분 8초 ~ 99분 17초**
- [이렇게 하세요] 상담 후 맞춤형 영양제를 제공받으세요.


In [11]:
# 출력 파싱, 주제와 시간 정보 추출
subtopics = re.findall(r'\*\*(\d+분 \d+초) ~ (\d+분 \d+초)\*\*\n- (.+?)(?=\n\*\*\d+분|\Z)', session_summary, re.DOTALL)

print(subtopics)


[('66분 8초', '66분 42초', 'Q. 고지혈증 약을 끊었는데, 영양제를 먹어야 할까요?\n'), ('67분 4초', '67분 19초', '[이렇게 하세요] 간 수치가 높지 않다면, 영양제를 드셔도 괜찮습니다.\n'), ('69분 10초', '69분 15초', '[이렇게 하세요] 피로감이 있다면, 영양제를 드시는 것이 좋습니다.\n'), ('69분 29초', '70분 40초', 'Q. 병원에서 처방받은 약과 약국에서 구입한 약의 차이는 무엇인가요?\n'), ('71분 5초', '71분 10초', '[바꾸세요] 감기 초기에 비타민을 섭취하여 면역력을 높이세요.\n'), ('72분 1초', '72분 17초', '[이렇게 하세요] 감기약은 증상에 맞춰서 드세요.\n'), ('74분 2초', '74분 22초', '[이렇게 하세요] 열이 나면 타이레놀을 드세요.\n'), ('87분 48초', '88분 26초', '[이렇게 하세요] 고지혈증 약을 오래 복용했다면 코엔자임 큐텐을 보충하세요.\n'), ('89분 10초', '89분 19초', '[이렇게 하세요] 저녁 식사를 간단히 하고 일찍 주무세요.\n'), ('92분 8초', '92분 50초', '[이렇게 하세요] 저녁 식사는 탄수화물보다 단백질과 채소 위주로 드세요.\n'), ('93분 0초', '93분 34초', '[바꾸세요] 술로 인해 잠을 자는 것은 피하세요.\n'), ('94분 8초', '94분 49초', '[이렇게 하세요] 잠들기 전에는 소음 없이 주무세요.\n'), ('97분 0초', '97분 11초', '[이렇게 하세요] 주말에는 낮잠을 통해 피로를 푸세요.\n'), ('98분 21초', '98분 34초', '[이렇게 하세요] 필요에 따라 영양제를 추천받아 드세요.\n'), ('99분 8초', '99분 17초', '[이렇게 하세요] 상담 후 맞춤형 영양제를 제공받으세요.')]


In [13]:
def time_str_to_ms(time_str):
    match = re.match(r'(\d+)분 (\d+)초', time_str)
    if match:
        minutes = int(match.group(1))
        seconds = int(match.group(2))
        return (minutes * 60 + seconds) * 1000
    else:
        logging.warning(f"시간 형식이 일치하지 않습니다: {time_str}")
        return 0
    
json_results = []
        
        # 주제별 루프 처리
for idx, (start_time_str, end_time_str, content) in enumerate(subtopics, 1):
    subtopic = content.strip()
    start_time_ms = time_str_to_ms(start_time_str)
    end_time_ms = time_str_to_ms(end_time_str)
    
    # 해당 시간 범위 내에서 retrieve
    retrieved_docs = retriever.get_relevant_documents(subtopic)
    
    relevant_docs = [
        doc for doc in retrieved_docs
        if start_time_ms <= doc.metadata['start_at'] <= end_time_ms
    ]
    
    # 관련 스크립트 생성
    related_scripts = [
        {
            "time": ms_to_minutes_seconds_str(doc.metadata['start_at']),
            "content": doc.page_content
        } for doc in relevant_docs
    ]
    
    # 주제별 데이터를 딕셔너리로 저장
    json_result = {
        "topic_id": idx,
        "start_time": start_time_str,
        "end_time": end_time_str,
        "content": subtopic,
        "related_scripts": related_scripts
    }
    json_results.append(json_result)
print(json_results)

C:\Users\yooji\AppData\Local\Temp\ipykernel_21116\3804599114.py:20: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_docs = retriever.get_relevant_documents(subtopic)
2024-11-08 15:27:57,718 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:27:57,990 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:27:58,475 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:27:58,841 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:27:59,465 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:27:59,955 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:28:00,507 - INFO - HTTP Request: POST https://a

[{'topic_id': 1, 'start_time': '66분 8초', 'end_time': '66분 42초', 'content': 'Q. 고지혈증 약을 끊었는데, 영양제를 먹어야 할까요?', 'related_scripts': [{'time': '66분 8초', 'content': '예, 보내주신 거는 미리 읽어보고 아는데요, 고지혈증 약만 드시고 계신 거죠.'}]}, {'topic_id': 2, 'start_time': '67분 4초', 'end_time': '67분 19초', 'content': '[이렇게 하세요] 간 수치가 높지 않다면, 영양제를 드셔도 괜찮습니다.', 'related_scripts': []}, {'topic_id': 3, 'start_time': '69분 10초', 'end_time': '69분 15초', 'content': '[이렇게 하세요] 피로감이 있다면, 영양제를 드시는 것이 좋습니다.', 'related_scripts': []}, {'topic_id': 4, 'start_time': '69분 29초', 'end_time': '70분 40초', 'content': 'Q. 병원에서 처방받은 약과 약국에서 구입한 약의 차이는 무엇인가요?', 'related_scripts': [{'time': '69분 54초', 'content': '근데 감기로만 비교를 하면 이제 그 약국에서 쓰는 약이나 병원, 약이나 차이는 그 한 가지예요.'}, {'time': '70분 28초', 'content': '예, 비슷한데 좀 그런 게 있고, 또 하나 차이는 병원에 가면 일단 직접 소독이 되잖아요.'}, {'time': '70분 21초', 'content': '되 그렇죠. 약 성분은 약국에서 그냥 파는 약이나 나머지 감기 뭐 콧물약은 비슷해요.'}, {'time': '69분 29초', 'content': '지금도 이제 간 게 기억 있는데 그냥 약국 가서 처약만 먹고 하거든요. 선생님, 근데 이게 병원에 가서 처방받는 게 나아요.'}]}, {'topic_

In [14]:
print(json_results[0])

{'topic_id': 1, 'start_time': '66분 8초', 'end_time': '66분 42초', 'content': 'Q. 고지혈증 약을 끊었는데, 영양제를 먹어야 할까요?', 'related_scripts': [{'time': '66분 8초', 'content': '예, 보내주신 거는 미리 읽어보고 아는데요, 고지혈증 약만 드시고 계신 거죠.'}]}


In [15]:
output_filename = 'json_results.json'

# Write json_results to a JSON file
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(json_results, f, ensure_ascii=False, indent=4)

print(f"JSON results saved to {output_filename}")

JSON results saved to json_results.json


Next steps:

to create docker and upload to lambda,
 docker buildx use mybuilder2
docker buildx build --platform linux/amd64 --load -t tenten_ai_topic .
docker tag tenten_ai_topic:latest 820604767531.dkr.ecr.ap-northeast-2.amazonaws.com/tenten_ai_topic:latest
docker push 820604767531.dkr.ecr.ap-northeast-2.amazonaws.com/tenten_ai_topic:latest

In [ ]:
If you use docker buildx, make sure to reuse the builder instance rather than creating a new one every time:
bash
Copy code
# Create the builder only once
docker buildx create --use --name mybuilder
Then, you can use the existing builder for subsequent builds:
bash
Copy code
docker buildx use mybuilder
docker buildx build --platform linux/amd64 --load -t tenten_ai_topic .
